In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import json

In [2]:
# Load the CSV file
def load_data(file_path):
    try:
        print(f"Loading data from {file_path}...")
        df = pd.read_csv(file_path)
        print(f"Successfully loaded {len(df)} records.")
        return df
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [3]:
# Load the data
file_path = "Data for Technical Challenge.csv"
df = load_data(file_path)

# Display the first few rows and data info to understand structure
if df is not None:
    print("Data Preview:")
    print(df.head())
    print("\nData Information:")
    print(df.info())
    print("\nData Statistics:")
    print(df.describe())
    print("\nMissing Values:")
    print(df.isnull().sum())

Loading data from Data for Technical Challenge.csv...
Successfully loaded 5706 records.
Data Preview:
  Date Invoiced   Date Paid  No. Days taken to Pay  Client Name  \
0    2022-05-31  2024-04-05                    675            1   
1    2023-08-16  2024-04-05                    233            1   
2    2023-08-16  2024-04-05                    233            1   
3    2023-08-16  2024-04-05                    233            1   
4    2019-01-31  2019-09-17                    229            1   

  Invoice Reference  Invoice Amount  Paid Amount  
0         2022-5916       105609.74    105609.74  
1         2023-6512       263821.73    263821.73  
2         2023-6512        90150.94     90150.94  
3         2023-6512         8442.53      8442.53  
4         2019-4610       102461.41    102461.40  

Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5706 entries, 0 to 5705
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------ 

In [4]:
def clean_data(df):
    # Create a copy to avoid modifying the original DataFrame
    cleaned_df = df.copy()
    
    # 1. Handle missing values
    print(f"Missing values before cleaning: {cleaned_df.isnull().sum().sum()}")
    
    # Fill missing numeric values with appropriate defaults
    if 'No. Days taken to Pay' in cleaned_df.columns:
        cleaned_df['No. Days taken to Pay'] = cleaned_df['No. Days taken to Pay'].fillna(0)
    
    if 'Invoice Amount' in cleaned_df.columns:
        cleaned_df['Invoice Amount'] = cleaned_df['Invoice Amount'].fillna(0)
    
    if 'Paid Amount' in cleaned_df.columns:
        cleaned_df['Paid Amount'] = cleaned_df['Paid Amount'].fillna(0)
    
    # 2. Convert date columns to datetime format
    date_columns = ['Date Invoiced', 'Date Paid']
    for col in date_columns:
        if col in cleaned_df.columns:
            # Try multiple date formats
            try:
                cleaned_df[col] = pd.to_datetime(cleaned_df[col], errors='coerce')
            except:
                print(f"Unable to convert {col} to datetime. Manual cleaning needed.")
    
    # 3. Fix the Invoice Reference format (should be YEAR-REFERENCE NUMBER)
    if 'Invoice Reference' in cleaned_df.columns:
        cleaned_df['Invoice Reference'] = cleaned_df['Invoice Reference'].fillna('Unknown')
        
        # Function to fix invoice reference format
        def fix_invoice_reference(ref):
            if isinstance(ref, str):
                # If already in YEAR-NUMBER format, return as is
                if re.match(r'^\d{4}-\d+$', ref):
                    return ref
                
                # Extract year from 'Date Invoiced' if available
                year = None
                invoice_date_idx = cleaned_df.columns.get_loc('Date Invoiced') if 'Date Invoiced' in cleaned_df.columns else None
                invoice_ref_idx = cleaned_df.columns.get_loc('Invoice Reference')
                
                if invoice_date_idx is not None:
                    row_idx = cleaned_df[cleaned_df['Invoice Reference'] == ref].index[0]
                    date_value = cleaned_df.iloc[row_idx, invoice_date_idx]
                    if isinstance(date_value, pd.Timestamp):
                        year = date_value.year
                
                # If year couldn't be extracted, use a default
                if year is None:
                    # Extract numbers from the reference
                    numbers = re.findall(r'\d+', ref)
                    if len(numbers) > 0 and len(numbers[0]) >= 4:
                        year = numbers[0][:4]  # Use first 4 digits as year
                    else:
                        year = "0000"  # Default value
                
                # Extract or create reference number
                ref_num = re.sub(r'[^\d]', '', ref)
                if not ref_num:
                    ref_num = "0000"
                
                return f"{year}-{ref_num}"
            return "0000-0000"  # Default for non-string values
        
        # Apply the function to fix references
        cleaned_df['Invoice Reference'] = cleaned_df.apply(
            lambda row: fix_invoice_reference(row['Invoice Reference']), 
            axis=1
        )
    
    # 4. Handle Client Name - ensure it's clean
    if 'Client Name' in cleaned_df.columns:
        # Convert to string if it's numeric
        if pd.api.types.is_numeric_dtype(cleaned_df['Client Name']):
            cleaned_df['Client Name'] = cleaned_df['Client Name'].astype(str)
        
        # Replace missing client names with 'Unknown'
        cleaned_df['Client Name'] = cleaned_df['Client Name'].fillna('Unknown')
    
    # 5. Check for duplicate invoices and handle them
    duplicates = cleaned_df.duplicated(subset=['Invoice Reference'], keep=False)
    if duplicates.any():
        print(f"Found {duplicates.sum()} duplicate invoice references.")
        # Keep the first occurrence for now, but flag them
        cleaned_df['Is Duplicate'] = duplicates
    
    # 6. Calculate or validate 'No. Days taken to Pay'
    if all(col in cleaned_df.columns for col in ['Date Invoiced', 'Date Paid']):
        # Calculate days to pay for rows with both dates
        has_both_dates = cleaned_df['Date Invoiced'].notna() & cleaned_df['Date Paid'].notna()
        cleaned_df.loc[has_both_dates, 'Calculated Days to Pay'] = (
            cleaned_df.loc[has_both_dates, 'Date Paid'] - 
            cleaned_df.loc[has_both_dates, 'Date Invoiced']
        ).dt.days
        
        # Check if calculated days match the provided 'No. Days taken to Pay'
        if 'No. Days taken to Pay' in cleaned_df.columns:
            days_match = (cleaned_df['Calculated Days to Pay'] == cleaned_df['No. Days taken to Pay'])
            mismatches = (~days_match) & cleaned_df['Calculated Days to Pay'].notna()
            
            if mismatches.any():
                print(f"Found {mismatches.sum()} mismatches in 'Days to Pay' calculation.")
                # Use calculated days where there are mismatches
                cleaned_df.loc[mismatches, 'No. Days taken to Pay'] = cleaned_df.loc[mismatches, 'Calculated Days to Pay']
            
            # Use calculated days where original is missing
            missing_days = cleaned_df['No. Days taken to Pay'].isna() & cleaned_df['Calculated Days to Pay'].notna()
            cleaned_df.loc[missing_days, 'No. Days taken to Pay'] = cleaned_df.loc[missing_days, 'Calculated Days to Pay']
        else:
            # If 'No. Days taken to Pay' doesn't exist, create it
            cleaned_df['No. Days taken to Pay'] = cleaned_df['Calculated Days to Pay']
    
    # 7. Add a payment status column
    if all(col in cleaned_df.columns for col in ['Invoice Amount', 'Paid Amount']):
        # Allow for small differences due to rounding
        tolerance = 0.01
        cleaned_df['Payment Status'] = 'Unknown'
        
        # Fully paid
        fully_paid = (abs(cleaned_df['Paid Amount'] - cleaned_df['Invoice Amount']) <= tolerance)
        cleaned_df.loc[fully_paid, 'Payment Status'] = 'Fully Paid'
        
        # Partially paid
        partially_paid = (cleaned_df['Paid Amount'] > 0) & (cleaned_df['Paid Amount'] < cleaned_df['Invoice Amount'] - tolerance)
        cleaned_df.loc[partially_paid, 'Payment Status'] = 'Partially Paid'
        
        # Unpaid
        unpaid = (cleaned_df['Paid Amount'] <= 0)
        cleaned_df.loc[unpaid, 'Payment Status'] = 'Unpaid'
        
        # Overpaid
        overpaid = (cleaned_df['Paid Amount'] > cleaned_df['Invoice Amount'] + tolerance)
        cleaned_df.loc[overpaid, 'Payment Status'] = 'Overpaid'
    
    # 8. Define Late Invoices (setting 30 days as threshold)
    if 'No. Days taken to Pay' in cleaned_df.columns:
        cleaned_df['Is Late'] = cleaned_df['No. Days taken to Pay'] > 30  # 30 days threshold
    
    # Clean up - drop temporary columns
    if 'Calculated Days to Pay' in cleaned_df.columns:
        cleaned_df = cleaned_df.drop(columns=['Calculated Days to Pay'])
    
    print(f"Missing values after cleaning: {cleaned_df.isnull().sum().sum()}")
    return cleaned_df

# Clean the data
cleaned_df = clean_data(df)
cleaned_df.head()

Missing values before cleaning: 0
Found 3746 duplicate invoice references.
Missing values after cleaning: 0


,Date Invoiced,Date Paid,No. Days taken to Pay,Client Name,Invoice Reference,Invoice Amount,Paid Amount,Is Duplicate,Payment Status,Is Late
0,2022-05-31,2024-04-05,675,1,2022-5916,105609.74,105609.74,False,Fully Paid,True
1,2023-08-16,2024-04-05,233,1,2023-6512,263821.73,263821.73,True,Fully Paid,True
2,2023-08-16,2024-04-05,233,1,2023-6512,90150.94,90150.94,True,Fully Paid,True
3,2023-08-16,2024-04-05,233,1,2023-6512,8442.53,8442.53,True,Fully Paid,True
4,2019-01-31,2019-09-17,229,1,2019-4610,102461.41,102461.40,True,Partially Paid,True


In [5]:
def add_derived_metrics(df):
    enriched_df = df.copy()
    
    # 1. Add month and year columns for aggregation
    if 'Date Invoiced' in enriched_df.columns and pd.api.types.is_datetime64_dtype(enriched_df['Date Invoiced']):
        enriched_df['Invoice Month'] = enriched_df['Date Invoiced'].dt.to_period('M')
        enriched_df['Invoice Year'] = enriched_df['Date Invoiced'].dt.year
    
    if 'Date Paid' in enriched_df.columns and pd.api.types.is_datetime64_dtype(enriched_df['Date Paid']):
        enriched_df['Payment Month'] = enriched_df['Date Paid'].dt.to_period('M')
        enriched_df['Payment Year'] = enriched_df['Date Paid'].dt.year
    
    # 2. Flag outstanding invoices (not paid or partially paid)
    if 'Payment Status' in enriched_df.columns:
        enriched_df['Is Outstanding'] = enriched_df['Payment Status'].isin(['Unpaid', 'Partially Paid'])
    
    # 3. Calculate amount due (Invoice Amount - Paid Amount)
    if all(col in enriched_df.columns for col in ['Invoice Amount', 'Paid Amount']):
        enriched_df['Amount Due'] = enriched_df['Invoice Amount'] - enriched_df['Paid Amount']
        # Ensure no negative values for Amount Due (handle overpayments)
        enriched_df['Amount Due'] = enriched_df['Amount Due'].clip(lower=0)
    
    return enriched_df

# Add derived metrics
enriched_df = add_derived_metrics(cleaned_df)
enriched_df.head()

,Date Invoiced,Date Paid,No. Days taken to Pay,Client Name,Invoice Reference,Invoice Amount,Paid Amount,Is Duplicate,Payment Status,Is Late,Invoice Month,Invoice Year,Payment Month,Payment Year,Is Outstanding,Amount Due
0,2022-05-31,2024-04-05,675,1,2022-5916,105609.74,105609.74,False,Fully Paid,True,2022-05,2022,2024-04,2024,False,0.00
1,2023-08-16,2024-04-05,233,1,2023-6512,263821.73,263821.73,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
2,2023-08-16,2024-04-05,233,1,2023-6512,90150.94,90150.94,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
3,2023-08-16,2024-04-05,233,1,2023-6512,8442.53,8442.53,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
4,2019-01-31,2019-09-17,229,1,2019-4610,102461.41,102461.40,True,Partially Paid,True,2019-01,2019,2019-09,2019,True,0.01


In [6]:
def calculate_company_metrics(df):
    # Get unique companies
    companies = df['Client Name'].unique()
    
    company_metrics = {}
    for company in companies:
        company_data = df[df['Client Name'] == company]
        
        # Calculate basic metrics for each company
        metrics = {
            'total_invoices': len(company_data),
            'total_invoiced': company_data['Invoice Amount'].sum(),
            'total_paid': company_data['Paid Amount'].sum(),
            'average_days_to_pay': company_data['No. Days taken to Pay'].mean(),
            'late_invoices_count': company_data['Is Late'].sum(),
            'late_invoices_percentage': (company_data['Is Late'].sum() / len(company_data)) * 100 if len(company_data) > 0 else 0,
            'outstanding_invoices': company_data['Is Outstanding'].sum(),
            'outstanding_amount': company_data['Amount Due'].sum()
        }
        
        # Calculate monthly totals
        if 'Invoice Month' in company_data.columns:
            monthly_invoiced = company_data.groupby('Invoice Month')['Invoice Amount'].sum()
            metrics['monthly_invoiced'] = monthly_invoiced.to_dict()
        
        if 'Payment Month' in company_data.columns:
            monthly_paid = company_data.groupby('Payment Month')['Paid Amount'].sum()
            metrics['monthly_paid'] = monthly_paid.to_dict()
        
        # Payment status breakdown
        if 'Payment Status' in company_data.columns:
            status_counts = company_data['Payment Status'].value_counts()
            metrics['payment_status_breakdown'] = status_counts.to_dict()
        
        company_metrics[company] = metrics
    
    return company_metrics

# Calculate company metrics
company_metrics = calculate_company_metrics(enriched_df)
company_metrics

{'1': {'total_invoices': 115,
  'total_invoiced': 30077153.9,
  'total_paid': 29533057.6,
  'average_days_to_pay': 93.98260869565217,
  'late_invoices_count': 102,
  'late_invoices_percentage': 88.69565217391305,
  'outstanding_invoices': 6,
  'outstanding_amount': 544096.3,
  'monthly_invoiced': {Period('2018-03', 'M'): 580549.48,
   Period('2018-05', 'M'): 727760.7100000001,
   Period('2018-06', 'M'): 354918.1,
   Period('2018-07', 'M'): 995112.84,
   Period('2018-09', 'M'): 468072.12,
   Period('2018-11', 'M'): 330246.28,
   Period('2019-01', 'M'): 1491623.93,
   Period('2019-02', 'M'): 57977.590000000004,
   Period('2019-04', 'M'): 75567.44,
   Period('2019-05', 'M'): 212833.72999999998,
   Period('2019-07', 'M'): 908150.22,
   Period('2019-08', 'M'): 60401.71,
   Period('2019-10', 'M'): 47239.799999999996,
   Period('2019-11', 'M'): 373207.88,
   Period('2019-12', 'M'): 504920.29,
   Period('2022-01', 'M'): 571605.74,
   Period('2022-02', 'M'): 982309.2100000001,
   Period('2022-0

In [7]:
import numpy as np

def numpy_to_python_type(obj):
    """Convert NumPy types to Python native types recursively."""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: numpy_to_python_type(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [numpy_to_python_type(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(numpy_to_python_type(item) for item in obj)
    else:
        return obj

In [8]:
def prepare_for_export(df, company_metrics):
    # 1. Convert to suitable formats for export
    export_df = df.copy()
    
    # Convert Period types to strings
    for col in ['Invoice Month', 'Payment Month']:
        if col in export_df.columns and pd.api.types.is_period_dtype(export_df[col]):
            export_df[col] = export_df[col].astype(str)
    
    # Convert datetime to ISO format strings
    for col in ['Date Invoiced', 'Date Paid']:
        if col in export_df.columns and pd.api.types.is_datetime64_dtype(export_df[col]):
            export_df[col] = export_df[col].dt.strftime('%Y-%m-%d')
    
    # 2. Prepare main invoice data for Supabase
    invoices_for_export = export_df.to_dict(orient='records')
    # Convert NumPy types in invoices
    invoices_for_export = numpy_to_python_type(invoices_for_export)
    
    # 3. Prepare company metrics for Supabase
    # Convert any non-serializable objects in company metrics
    company_metrics_export = {}
    for company, metrics in company_metrics.items():
        company_metrics_export[company] = {}
        for key, value in metrics.items():
            if key in ['monthly_invoiced', 'monthly_paid']:
                # Convert Period keys to strings and values to Python types
                company_metrics_export[company][key] = {
                    str(k): numpy_to_python_type(v) for k, v in value.items()
                }
            else:
                # Convert metrics values to Python types
                company_metrics_export[company][key] = numpy_to_python_type(value)
    
    return {
        'invoices': invoices_for_export,
        'company_metrics': company_metrics_export
    }
    
# Prepare data for export
export_data = prepare_for_export(enriched_df, company_metrics)

# Export to JSON files
def export_to_json(data, prefix='cleaned_data'):
    try:
        with open(f"{prefix}_invoices.json", 'w') as f:
            json.dump(data['invoices'], f)
        
        with open(f"{prefix}_company_metrics.json", 'w') as f:
            json.dump(data['company_metrics'], f)
        
        print(f"Successfully exported data to {prefix}_invoices.json and {prefix}_company_metrics.json")
    except Exception as e:
        print(f"Error exporting data: {e}")

# Export the data
export_to_json(export_data)

# Also save to CSV for alternative usage
enriched_df.to_csv('cleaned_invoice_data.csv', index=False)
print("Successfully exported data to cleaned_invoice_data.csv")

Successfully exported data to cleaned_data_invoices.json and cleaned_data_company_metrics.json
Successfully exported data to cleaned_invoice_data.csv


/var/folders/xx/bt0lfdqx57584v3465p420c80000gn/T/ipykernel_84846/3781078052.py:7: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if col in export_df.columns and pd.api.types.is_period_dtype(export_df[col]):
/var/folders/xx/bt0lfdqx57584v3465p420c80000gn/T/ipykernel_84846/3781078052.py:7: DeprecationWarning: is_period_dtype is deprecated and will be removed in a future version. Use `isinstance(dtype, pd.PeriodDtype)` instead
  if col in export_df.columns and pd.api.types.is_period_dtype(export_df[col]):


In [9]:
new_df = pd.read_csv("cleaned_invoice_data.csv")
new_df.head()

,Date Invoiced,Date Paid,No. Days taken to Pay,Client Name,Invoice Reference,Invoice Amount,Paid Amount,Is Duplicate,Payment Status,Is Late,Invoice Month,Invoice Year,Payment Month,Payment Year,Is Outstanding,Amount Due
0,2022-05-31,2024-04-05,675,1,2022-5916,105609.74,105609.74,False,Fully Paid,True,2022-05,2022,2024-04,2024,False,0.00
1,2023-08-16,2024-04-05,233,1,2023-6512,263821.73,263821.73,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
2,2023-08-16,2024-04-05,233,1,2023-6512,90150.94,90150.94,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
3,2023-08-16,2024-04-05,233,1,2023-6512,8442.53,8442.53,True,Fully Paid,True,2023-08,2023,2024-04,2024,False,0.00
4,2019-01-31,2019-09-17,229,1,2019-4610,102461.41,102461.40,True,Partially Paid,True,2019-01,2019,2019-09,2019,True,0.01


In [10]:
try:
    with open('cleaned_data_company_metrics.json', 'r') as f:
        company_metrics = json.load(f)
    
    print(f"\nCompany Metrics JSON: Successfully loaded metrics for {len(company_metrics)} companies")
    
    # Basic validation
    if len(company_metrics) > 0:
        print("  ✅ Company metrics data contains records")
        
        # Show sample of first company
        company_name = list(company_metrics.keys())[0]
        print(f"\nSample metrics for company '{company_name}':")
        metrics = company_metrics[company_name]
        for key, value in list(metrics.items())[:10]:  # Show first 10 metrics
            if isinstance(value, dict):
                print(f"  {key}: {type(value)} with {len(value)} entries")
            else:
                print(f"  {key}: {value}")
        
        # Check for required metrics
        required_metrics = ['total_invoices', 'average_days_to_pay', 'late_invoices_count']
        missing_metrics = [metric for metric in required_metrics if metric not in metrics]
        
        if missing_metrics:
            print(f"\n  ⚠️ Missing required metrics: {missing_metrics}")
        else:
            print("\n  ✅ All required metrics are present")
    else:
        print("  ⚠️ No company metrics found in the file")
except Exception as e:
    print(f"  ❌ Error loading company metrics JSON: {e}")


Company Metrics JSON: Successfully loaded metrics for 280 companies
  ✅ Company metrics data contains records

Sample metrics for company '1':
  total_invoices: 115
  total_invoiced: 30077153.9
  total_paid: 29533057.6
  average_days_to_pay: 93.98260869565217
  late_invoices_count: 102
  late_invoices_percentage: 88.69565217391305
  outstanding_invoices: 6
  outstanding_amount: 544096.3
  monthly_invoiced: <class 'dict'> with 40 entries
  monthly_paid: <class 'dict'> with 36 entries

  ✅ All required metrics are present


In [11]:
try:
    with open('cleaned_data_invoices.json', 'r') as f:
        invoices_data = json.load(f)
    
    print(f"\nInvoices JSON: Successfully loaded {len(invoices_data)} records")
    
    # Basic validation
    if len(invoices_data) > 0:
        print("  ✅ Invoice data contains records")
        
        # Show sample of first invoice
        print("\nSample invoice record:")
        for key, value in list(invoices_data[0].items())[:10]:  # Show first 10 fields
            print(f"  {key}: {value}")
        
        # Check for required fields
        required_fields = ['Invoice Reference', 'Client Name', 'Invoice Amount', 'Paid Amount']
        missing_fields = [field for field in required_fields if field not in invoices_data[0]]
        
        if missing_fields:
            print(f"\n  ⚠️ Missing required fields: {missing_fields}")
        else:
            print("\n  ✅ All required fields are present")
    else:
        print("  ⚠️ No invoice records found in the file")
except Exception as e:
    print(f"  ❌ Error loading invoices JSON: {e}")


Invoices JSON: Successfully loaded 5706 records
  ✅ Invoice data contains records

Sample invoice record:
  Date Invoiced: 2022-05-31
  Date Paid: 2024-04-05
  No. Days taken to Pay: 675
  Client Name: 1
  Invoice Reference: 2022-5916
  Invoice Amount: 105609.74
  Paid Amount: 105609.74
  Is Duplicate: False
  Payment Status: Fully Paid
  Is Late: True

  ✅ All required fields are present


In [12]:
import pandas as pd
import json
import uuid
import os

def add_primary_key_to_invoice_csv(input_file, output_file):
    """Add a UUID primary key to the invoice CSV file."""
    print(f"Processing invoice data from {input_file}...")
    
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Add a UUID primary key column
    df['id'] = [str(uuid.uuid4()) for _ in range(len(df))]
    
    # Move id column to the front
    cols = df.columns.tolist()
    cols = ['id'] + [col for col in cols if col != 'id']
    df = df[cols]
    
    # Write to new CSV file
    df.to_csv(output_file, index=False)
    print(f"Successfully created {output_file} with {len(df)} records")
    
    return df

def convert_company_metrics_to_csv(input_file, output_file):
    """Convert company metrics JSON to CSV with primary key."""
    print(f"Converting company metrics from {input_file} to CSV...")
    
    # Read the JSON file
    with open(input_file, 'r') as f:
        metrics_data = json.load(f)
    
    # Create records with flattened structure
    records = []
    for company_name, metrics in metrics_data.items():
        # Extract basic metrics
        record = {
            'id': str(uuid.uuid4()),
            'client_name': company_name,
            'total_invoices': metrics.get('total_invoices', 0),
            'total_invoiced': metrics.get('total_invoiced', 0),
            'total_paid': metrics.get('total_paid', 0),
            'average_days_to_pay': metrics.get('average_days_to_pay', 0),
            'late_invoices_count': metrics.get('late_invoices_count', 0),
            'late_invoices_percentage': metrics.get('late_invoices_percentage', 0),
            'outstanding_invoices': metrics.get('outstanding_invoices', 0),
            'outstanding_amount': metrics.get('outstanding_amount', 0)
        }
        
        # Handle nested JSON structures - convert to string representation
        monthly_data = {}
        if 'monthly_invoiced' in metrics:
            monthly_data['monthly_invoiced'] = metrics['monthly_invoiced']
        if 'monthly_paid' in metrics:
            monthly_data['monthly_paid'] = metrics['monthly_paid']
            
        record['monthly_data'] = json.dumps(monthly_data) if monthly_data else None
        record['payment_status_breakdown'] = json.dumps(metrics.get('payment_status_breakdown', {}))
        
        records.append(record)
    
    # Create DataFrame and write to CSV
    df = pd.DataFrame(records)
    df.to_csv(output_file, index=False)
    print(f"Successfully created {output_file} with {len(df)} records")
    
    return df

def create_monthly_breakdown_csv(invoice_df, output_file):
    """Create a separate CSV for monthly breakdown data with a relation to company."""
    print("Creating monthly breakdown CSV for detailed time-series analysis...")
    
    # Group by client name and month to get monthly totals
    monthly_records = []
    
    # Ensure we have the needed columns
    if all(col in invoice_df.columns for col in ['Client Name', 'Invoice Month', 'Invoice Amount']):
        # Get unique client-month combinations
        grouped = invoice_df.groupby(['Client Name', 'Invoice Month']).agg({
            'Invoice Amount': 'sum',
            'Paid Amount': 'sum'
        }).reset_index()
        
        # Create records with proper relations
        for _, row in grouped.iterrows():
            monthly_records.append({
                'id': str(uuid.uuid4()),
                'client_name': row['Client Name'],
                'month': row['Invoice Month'],
                'invoiced_amount': row['Invoice Amount'],
                'paid_amount': row['Paid Amount']
            })
        
        # Create DataFrame and write to CSV
        monthly_df = pd.DataFrame(monthly_records)
        monthly_df.to_csv(output_file, index=False)
        print(f"Successfully created {output_file} with {len(monthly_df)} records")

def main():
    # Input and output file paths
    invoice_input = "cleaned_invoice_data.csv"
    company_metrics_input = "cleaned_data_company_metrics.json"
    
    invoice_output = "supabase_invoices.csv"
    company_metrics_output = "supabase_company_metrics.csv"
    monthly_breakdown_output = "supabase_monthly_breakdown.csv"
    
    # Process invoice data
    invoice_df = add_primary_key_to_invoice_csv(invoice_input, invoice_output)
    
    # Process company metrics
    convert_company_metrics_to_csv(company_metrics_input, company_metrics_output)
    
    # Create monthly breakdown CSV (optional but useful for time-series analysis)
    create_monthly_breakdown_csv(invoice_df, monthly_breakdown_output)
    
    print("\nAll files have been processed successfully!")
    print("You can now directly import these CSV files to Supabase:")
    print(f"1. {invoice_output} - Main invoice data with primary keys")
    print(f"2. {company_metrics_output} - Company metrics with primary keys")
    print(f"3. {monthly_breakdown_output} - Monthly breakdown data for time-series visualization")

if __name__ == "__main__":
    main()

Processing invoice data from cleaned_invoice_data.csv...
Successfully created supabase_invoices.csv with 5706 records
Converting company metrics from cleaned_data_company_metrics.json to CSV...
Successfully created supabase_company_metrics.csv with 280 records
Creating monthly breakdown CSV for detailed time-series analysis...
Successfully created supabase_monthly_breakdown.csv with 2313 records

All files have been processed successfully!
You can now directly import these CSV files to Supabase:
1. supabase_invoices.csv - Main invoice data with primary keys
2. supabase_company_metrics.csv - Company metrics with primary keys
3. supabase_monthly_breakdown.csv - Monthly breakdown data for time-series visualization
